In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import (accuracy_score,
                             precision_score,
                             recall_score,
                             f1_score,
                             confusion_matrix,
                             classification_report,
                             ConfusionMatrixDisplay)

path = "/content/drive/MyDrive/ML_Hillel/HW_L13_work_with_text/IMDB Dataset/"

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
'''
path1 = '/content/drive/MyDrive/ML_Hillel/HW_L13_work_with_text/IMDB Dataset/IMDB Dataset.csv'
reviews = open(path1, 'r')

header = reviews.readline()[:-1].split(',')
print(header)

reviews_data = []
for line in reviews:
    row = re.split(r'"([^"]+)",\s*', line)
    reviews_data.append(row)

print(reviews_data[:2])
print(len(reviews_data))

# Закриття файлу
reviews.close()
'''


'\npath1 = \'/content/drive/MyDrive/ML_Hillel/HW_L13_work_with_text/IMDB Dataset/IMDB Dataset.csv\'\nreviews = open(path1, \'r\')\n\nheader = reviews.readline()[:-1].split(\',\')\nprint(header)\n\nreviews_data = []\nfor line in reviews:\n    row = re.split(r\'"([^"]+)",\\s*\', line)\n    reviews_data.append(row)\n\nprint(reviews_data[:2])\nprint(len(reviews_data))\n\n# Закриття файлу\nreviews.close()\n'

In [5]:
# Зчитуємо датасет :
df = pd.read_csv(path + 'IMDB Dataset.csv', header=0, quotechar='"')
df

review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]

In [6]:
print(df.shape)
df['sentiment'].value_counts()

(50000, 2)


sentiment
positive    25000
negative    25000
Name: count, dtype: int64

Датасет успішно вдалося завантажити і розділити на 2 колонки.

Звернемо увагу на рівномірність розподілу таргету

##Preprocessing

In [7]:
#Позбудемося зайвих пробілів на краях
df['review'] = df['review'].str.strip()

# Видаляємо HTML-теги за допомогою регулярного виразу
df['review'] = df['review'].apply(lambda x: re.sub(r'<.*?>', '', x))

df

review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. The filming tec...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]

##Bag of words

In [8]:
#Перетворимо значення текстів за допомогою CountVectorizer, використавши словник стоп-слів
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['review'])

In [9]:
X.shape

(50000, 103771)

В результаті перетворення кількість колонок перевищило кількість об'єктів

In [10]:
#Виведемо список можливих значень слів
vectorizer.get_feature_names_out()

array(['00', '000', '00000000000', ..., 'żmijewski', 'יגאל', 'כרמון'],
      dtype=object)

In [11]:
y = df['sentiment']

In [12]:
# Для подальшого застосування моделей перетворимо таргет у числа
y = y.apply(lambda x: 1 if (x == 'positive') else 0 )
y

0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64

Далі побудуємо модель класифікації за допомогою RandomForest

In [12]:
# Поділ на тренувальні та тестові дані
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17)

#Для моделювання виберемо RandomForest

# Навчання класифікатора
rf_clf = RandomForestClassifier(random_state=17)
rf_clf.fit(X_train, y_train)

#Прогноз моделі
y_pred = rf_clf.predict(X_test)

#Вивід значень оцінок моделі
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred))
print('Recall:', recall_score(y_test, y_pred))
print('F1:', f1_score(y_test, y_pred))

Accuracy: 0.8608
Precision: 0.868151715833835
Recall: 0.8547717842323651
F1: 0.8614097968936678


З результатів бачимо, що модель показала досить високі результати по всіх показниках, отже, запропонований підхід вважаємо успішний

## TF_IDF

1.Через TfidfVectorizer

In [13]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
X_tfidf_vectorizer = tfidf_vectorizer.fit_transform(df['review'])

In [14]:
X_tfidf_vectorizer.shape

(50000, 103771)

In [15]:
#Виведемо список можливих значень слів
vectorizer.get_feature_names_out()

array(['00', '000', '00000000000', ..., 'żmijewski', 'יגאל', 'כרמון'],
      dtype=object)

Далі побудуємо модель класифікації за допомогою RandomForest (TfidfVectorizer)

In [16]:
# Поділ на тренувальні та тестові дані
X_train, X_test, y_train, y_test = train_test_split(X_tfidf_vectorizer, y, test_size=0.2, random_state=17)

#Для моделювання виберемо RandomForest

# Навчання класифікатора
rf_clf = RandomForestClassifier(random_state=17)
rf_clf.fit(X_train, y_train)

#Прогноз моделі
y_pred = rf_clf.predict(X_test)

#Вивід значень оцінок моделі
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred))
print('Recall:', recall_score(y_test, y_pred))
print('F1:', f1_score(y_test, y_pred))

Accuracy: 0.8642
Precision: 0.8715633152719245
Recall: 0.8581308041888954
F1: 0.864794902429311


2. Через композицію TfidfTransformer o CountVectorizer (перевіримо чи застосування даного методу ідентичне TfidfVectorizer)

In [16]:
#Для композиції використаємо Pipeline
pipe = Pipeline([('count', CountVectorizer(stop_words='english')), ('tfid', TfidfTransformer())]).fit_transform(df['review'])

In [17]:
pipe.shape

(50000, 103771)

Далі побудуємо модель класифікації за допомогою RandomForest

In [18]:
# Поділ на тренувальні та тестові дані
X_train, X_test, y_train, y_test = train_test_split(pipe, y, test_size=0.2, random_state=17)

#Для моделювання виберемо RandomForest

# Навчання класифікатора
rf_clf = RandomForestClassifier(random_state=17)
rf_clf.fit(X_train, y_train)

#Прогноз моделі
y_pred = rf_clf.predict(X_test)

#Вивід значень оцінок моделі
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred))
print('Recall:', recall_score(y_test, y_pred))
print('F1:', f1_score(y_test, y_pred))

Accuracy: 0.8642
Precision: 0.8715633152719245
Recall: 0.8581308041888954
F1: 0.864794902429311


Як бачимо, результати моделей RandomForest згідно перетворень TfidfVectorizer та композиції TfidfTransformer o CountVectorizer абсолютно ідентичні; переконалися, що ці перетворення тотожні.

Висновок: застосував перетворення тексту за допомогою Bag of words (CountVectorizer) i TfidfTransformer o CountVectorizer (TfidfVectorizer), після чого побудував базову модедь класифікації за RandomForest, яка показала високий рівень всіх оцінок (>85%)